In [2]:
using Statistics
using Metrics
using Knet

┌ Error: Cannot use NVML, as it failed to initialize
│   exception = (ErrorException("could not load symbol \"nvmlInitWithFlags\":\nThe specified procedure could not be found. "), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00000000026948e5, Ptr{Nothing} @0x0000000002723c09, Ptr{Nothing} @0x00000000027241e7, Ptr{Nothing} @0x0000000002694324, Ptr{Nothing} @0x00000000143ecd24, Ptr{Nothing} @0x00000000143ed0e3, Ptr{Nothing} @0x00000000143e08ce, Ptr{Nothing} @0x00000000143e09f7, Ptr{Nothing} @0x000000000273f1e0, Ptr{Nothing} @0x00000000027357b0, Ptr{Nothing} @0x00000000143c78ac, Ptr{Nothing} @0x00000000143c8b46, Ptr{Nothing} @0x00000000143cad09, Ptr{Nothing} @0x00000000143ccde3, Ptr{Nothing} @0x00000000143cdac3, Ptr{Nothing} @0x000000000273ec66, Ptr{Nothing} @0x000000000274098c, Ptr{Nothing} @0x0000000002740561, Ptr{Nothing} @0x00000000027414af, Ptr{Nothing} @0x00000000143bc565, Ptr{Nothing} @0x00000000143bc93f, Ptr{Nothing} @0x00000000143b9528, Ptr{Nothing} @0x0000000002716186

In [1]:
# Reading data for sentiment classification
# it's already divided into train/dev/test

function read_sentiment_data(file_path)
    labels = []
    data = []
    try
        open(file_path) do f
            while ! eof(f)      
                s = readline(f) 
                s = lowercase.(s)
                s = replace.(s, r"[,.:;?!()]" => "")
                s_with_spaces = replace(s, '\t' => ' ') 
                temp = split(s_with_spaces," ")
                
                push!(labels,temp[end-1])
                push!(data,temp[2:end-2])
            end
        end
    catch 
        println("file doesn't exist")
    end
    return data, labels
end

path = "test.txt"
#path = raw"C:\Users\brkcn\HIT-ACL2021-Codemixed-Representation\data\malayalam\malayalam_train.tsv"
#path = raw"C:\Users\brkcn\HIT-ACL2021-Codemixed-Representation\data\tamil\tamil_train.tsv"
#path = raw"C:\Users\brkcn\HIT-ACL2021-Codemixed-Representation\data\hindi_sentiment\IIITH_Codemixed.txt"
sentiment_data, sentiment_labels = read_sentiment_data(path);



In [4]:
# Reading data for machine translation
# it's already divided into train/dev/test
function read_mt_data(file_path)
    data = []
    try
        open(file_path) do f
            while ! eof(f)      
                s = readline(f) 
                s = lowercase.(s)
                s = replace.(s, r"[,.:;?!()]" => "")
                s = replace(s, '\t' => ' ') 
                s = split(s," ")
                push!(data,s)
            end
        end
    catch 
        println("file doesn't exist")
    end
    data
end

path = raw"C:\Users\brkcn\HIT-ACL2021-Codemixed-Representation\data\IITPatna-CodeMixedMT\train.src"
mt_data= read_mt_data(path);


In [5]:
# Reading data for pos tagging
function read_pos_tagging_data(file_path)
    words = []
    langs = []
    tags = []
    try
        open(file_path) do f
            while ! eof(f)
                s = readline(f) 
                if length(s) != 0
                    if s[1] != '@'
                        s = replace(s, '\t' => ' ') 
                        s = split(s," ")
                        push!(words,s[1])
                        push!(langs,s[2])
                        push!(tags,s[3])
                    end
                end
            end
        end
    catch 
        println("error during reading pos tagging data")
    end
    words, langs, tags
end
path = raw"C:\Users\brkcn\HIT-ACL2021-Codemixed-Representation\data\bengali_POS\TWT_BN_EN_FN.txt"
pos_word, pos_langs, pos_tags = read_pos_tagging_data(path);



In [6]:
#Randomly predcting for now
function pred_sentiment(X)
    preds = []
    for sentence in X
        push!(preds,rand(1:4))
    end
    preds
end


function vec_labels(y)
    num_labels = []
    for label in y
        if label == "positive"
            push!(num_labels,1)
        elseif label == "negative"
            push!(num_labels,2)
        elseif label == "mixed_feelings"
            push!(num_labels,3)
        else
            push!(num_labels,4)
        end
    end
    num_labels
end



vec_labels (generic function with 1 method)

In [35]:
function calculate_metrics(pred,y,num_classes)
    y_len = length(y)
    conf_mat = zeros(Int16,num_classes,num_classes)
    for i in 1:y_len
        conf_mat[pred[i],y[i]] += 1
    end
    positives = []
    for j in 1:num_classes
        push!(positives,conf_mat[j,j])
    end
    recall = positives./sum(conf_mat,dims=1)'
    precision = positives./sum(conf_mat,dims=2)
    f1_score =  2*(precision.*recall)./(precision.+recall)
    mean(f1_score),mean(precision),mean(recall)
end


y = vec_labels(sentiment_labels)
pred_y  = pred_sentiment(sentiment_data)
sentiment_f1_score, sentiment_precision, sentiment_recall =  calculate_metrics(pred_y,y,4)



println("sentiment f1 score: $sentiment_f1_score")
println("sentiment precision: $sentiment_precision")
println("sentiment sentiment recall: $sentiment_recall")

sentiment f1 score: 0.21899710633845815
sentiment precision: 0.24714519576778163
sentiment sentiment recall: 0.2553106750175261


In [42]:
# Testing bleu and rogue metrics for machine translation task later on.

ref_corpus = [["Example of bleu score"], ["This is an apple"]]
translated_corpus = ["Example to bleu score", "This no a apple"]
bleu_result = bleu_score(ref_corpus,translated_corpus)
println("bleu score: $(bleu_result[1])")

hypothesis = ["Example for bleu score", "This cz an apple"]
ref_corpus = ["Example of bleu score", "This is an apple"]
println("rogue score dict:")
rouge_out = rouge(hypothesis, ref_corpus)


bleu score: 0.7253666236200924
rogue score dict:


OrderedCollections.OrderedDict{String, Float64} with 9 entries:
  "rouge_1 / f_score" => 0.75
  "rouge_1 / r_score" => 0.75
  "rouge_1 / p_score" => 0.75
  "rouge_2 / f_score" => 0.333333
  "rouge_2 / r_score" => 0.333333
  "rouge_2 / p_score" => 0.333333
  "rouge_l / f_score" => 0.75
  "rouge_l / r_score" => 0.75
  "rouge_l / p_score" => 0.75

In [28]:
#generating word embeddings 

path = raw"C:\Users\brkcn\HIT-ACL2021-Codemixed-Representation\data\malayalam\malayalam_train.tsv"
sentiment_data, sentiment_labels = read_sentiment_data(path);

function generate_word_embeddings(sentiment_data,embedding_space=150)
    word_counts = Dict()

    for sentence in sentiment_data
        for word in sentence
           word_counts[word] = get(word_counts, word, 0) + 1
        end
    end

    sorted_word_counts = sort(collect(word_counts), by=x->x[2],rev=true)

    word_symbols = Dict([("<unk>", 1), ("<s>", 2),("<pad>",3)])
    for (index,word_n_count ) in enumerate(sorted_word_counts)
        word_symbols[word_n_count[1]] = index + 3
    end
    embedding_count = length(word_symbols)
    word_embeddings =  rand(embedding_count,embedding_space)
    
    sorted_word_counts, word_embeddings, word_symbols
end

sorted_word_counts, word_embeddings, word_symbols = generate_word_embeddings(sentiment_data)
word_embeddings




13563×150 Matrix{Float64}:
 0.305849   0.773973   0.422861   …  0.532339   0.527935   0.1201
 0.590452   0.428254   0.886631      0.880701   0.53276    0.151884
 0.109101   0.0890532  0.37157       0.273779   0.0362677  0.688134
 0.878168   0.863311   0.279708      0.0534905  0.0217467  0.120134
 0.752377   0.405445   0.104159      0.179468   0.420645   0.0227377
 0.134298   0.155499   0.129335   …  0.524802   0.918577   0.727608
 0.359881   0.689466   0.563064      0.0850109  0.919817   0.361457
 0.132755   0.367557   0.0818739     0.487579   0.738634   0.742328
 0.613248   0.25118    0.156225      0.0865275  0.743104   0.298876
 0.547914   0.611622   0.291513      0.298254   0.981845   0.941216
 0.0663833  0.426276   0.513892   …  0.565518   0.72773    0.281176
 0.953749   0.864944   0.961623      0.858811   0.96747    0.938396
 0.0443058  0.649013   0.165217      0.6621     0.797268   0.397158
 ⋮                                ⋱                        
 0.485291   0.519219   0.03568

In [64]:
# adding pad token to have fixed length input.

path = raw"C:\Users\brkcn\HIT-ACL2021-Codemixed-Representation\data\malayalam\malayalam_train.tsv"
#path = "test.txt"
sentiment_data, sentiment_labels = read_sentiment_data(path);

function pad_data(sentiment_data,max_len = 15)
    padded_sentiment_data = []
    for sentence in sentiment_data
        new_sentence = sentence
        if length(sentence) > max_len
            new_sentence = sentence[1:max_len]
        elseif length(sentence) < max_len
            pad_tokens =  repeat(["<pad>"], max_len - length(sentence))
            new_sentence = vcat(sentence,pad_tokens)
        end  
        push!(padded_sentiment_data,new_sentence)
    end
    padded_sentiment_data
end

pad_data(sentiment_data,12)


4852-element Vector{Any}:
 AbstractString["<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>"]
 AbstractString["hoo", "mammokka", "police", "vesham", "aaha", "anthas", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>"]
 AbstractString["oru", "rekshayum", "illakidilam", "kannu", "nananjupoyi", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>"]
 AbstractString["ikka", "", "", "", "", "waiting", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>"]
 AbstractString["raju", "ettante", "oro", "shorttum", "ijathi", "ppwli", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>"]
 AbstractString["ettan", "fansil", "netti", "poya", "aarenkilum", "undo", "", "", "", "#maduraraja#waiting", "#", "<pad>"]
 AbstractString["waiting", "to", "see", "mammukaas", "undaa", "🤣", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>", "<pad>"]
 SubString{String}["last", "aa", "chadi", "adi", "uff", "", "", "", "", "", "", ""]
 AbstractString["1", "day